In [ ]:
import argparse
import datetime
import logging
import sys
from pathlib import Path
import time

import numpy as np
import torch
import yaml

torch.cuda.is_available()
import os
# print(torch.cuda.get_device_name(1))
from tensorboardX import SummaryWriter
from tqdm import tqdm
from lib.dbscan_utils import validate
from model.encoders import PointNet, DGCNN, SCGEncoder
from model.model import MLP, SiamCluster
from dataset.dataloader import data_loader
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
config = yaml.load(open("config/config_dim_red.yaml"),Loader=yaml.FullLoader)

In [ ]:
config

In [ ]:
train_data_loader, val_data_loader = data_loader(config)

In [ ]:
with torch.cuda.device(config['util']['gpu']):
        dims = config['model']['dims']
        # print(torch.cuda.current_device())
        l_dim = dims
        if config['model']['md'] == 'pn':
            net = PointNet(dims=config['model']['dims'], is_normal=False, feature_transform=True)
        elif config['model']['md'] == 'scg':
            dims = 512 * 3
            l_dim = 512 * 2
            net = SCGEncoder(last_dims=config['model']['dims'], is_normal=config['training']['normal'], n_rkhs=512)
        elif config['model']['md'] == 'dg':
            net = DGCNN(dims=dims, k=config['model']['neighs'])
        else:
            raise NotImplementedError
        net = net.cuda()
        projector = MLP(in_size=dims, out_size=config['model']['proj_dim']).cuda()
        predictor = MLP(in_size=config['model']['proj_dim'], out_size=config['model']['proj_dim'], hidden_size=512, used='pred').cuda()
        # decoder = FoldingNet(dims, k=32).cuda()
        # decoder = DecoderFC(latent_dim=dims, output_pts=config['dataset']['num_]).cuda()
        decoder = None
        ema_net = SiamCluster(net, projector, predictor, dim=l_dim, clusters=config['model']['K'],
                            tau=config['model']['tau'], l_type=config['training']['l_type'], decoder=decoder).cuda()

In [ ]:
try:
    checkpoint = torch.load(config['model']['path'])
    ema_net.load_state_dict(checkpoint['model_state_dict'])
    print('Use pretrain model')
except Exception as e:
    print('No existing model, starting training from scratch {}'.format(e))

In [ ]:
pca = PCA(n_components=config['pca']['n_components'])
tsne = TSNE(n_components=config['tsne']['n_components'])

In [ ]:
all_points = []
for batch_id, val_data in tqdm(enumerate(val_data_loader, 0), total=len(val_data_loader), smoothing=0.9,desc = 'validation dataset'):
                x1_val, x2_val = val_data
                x1_val = x1_val.cuda()
                x1_val = x1_val.transpose(2, 1)
                x2_val = x2_val.cuda()
                x2_val = x2_val.transpose(2, 1)
                emb =  ema_net(x1_val,return_embedding=config['training']['return_embedding'])
                all_points.append(emb[0].detach().cpu().numpy())

In [ ]:
all_points = np.concatenate(all_points,axis=0)

In [ ]:
all_points.shape

In [21]:
x1_pca = pca.fit_transform(all_points)
print(x1_pca.shape)
x1_tsne = tsne.fit_transform(x1_pca)
print(x1_tsne.shape)

In [ ]:
config['training']['return_embedding']

In [ ]:
config['util']['gpu']